In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch as pt

In [7]:
# read the data from the text file
file_path = 'prices.txt'
data = pd.read_csv(file_path, sep='\s+', header=None)